In [2]:
import pandas as pd
import numpy as np
from typing import Union

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
full_df = pd.read_csv('/content/drive/MyDrive/rec_sys/train.csv')
full_df

,idx
0,333396 267089 155959 353335 414000 339989 2741...
1,174197 335779 141676 119856 376664 311755 3881...
2,254861 213397 462645 403619 446211 221833 1572...
3,132371 199878 357751 92032 94315 435049 260359...
4,408028 326575 425054 152422 387296 382570 3505...
...,...
1160079,253016 99116 345735 279348 213245 66599 182745...
1160080,304079 68730 203881 127902 12698 473137 244723...
1160081,92515 75156 480685 303888 268018 24493 142752 ...
1160082,334264 438056 313741 78977 314239 445180 46939...


In [11]:
data = pd.DataFrame(columns=['user', 'item', 'order'])

In [9]:
full_df = full_df.idx.astype('str')

In [12]:
for idx, rows in full_df[:500].T.iteritems():
  arr = rows.split(' ')
  rank = len(arr)
  for j in arr:
    rank-=1
    new_row = {'user': idx, 'item': j, 'order': rank}
    data = data.append(new_row, ignore_index=True)
data

,user,item,order
0,0,333396,53
1,0,267089,52
2,0,155959,51
3,0,353335,50
4,0,414000,49
...,...,...,...
38110,499,356082,4
38111,499,35274,3
38112,499,364777,2
38113,499,174780,1


In [19]:
data

,user,item,order
0,0,333396,53
1,0,267089,52
2,0,155959,51
3,0,353335,50
4,0,414000,49
...,...,...,...
38110,499,356082,4
38111,499,35274,3
38112,499,364777,2
38113,499,174780,1


In [20]:
class UsersKFoldPOut():
    def __init__(self, n_folds, p, random_seed=23, user_column='user', order='order'):
        self.n_folds = n_folds
        self.p = p
        self.random_seed = random_seed
        self.user_column = user_column
        self.order_column = order
        
    
    def split(self, df):
        users = df[self.user_column].unique()
        users_count = len(users)
        
        np.random.seed(self.random_seed)
        np.random.shuffle(users)
        
        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)
        fold_sizes[: users_count % self.n_folds] += 1
        current = 0

        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            test_mask = df[self.user_column].isin(test_fold_users) & (df[self.order_column] < self.p)
            train_mask = ~df[self.user_column].isin(test_fold_users)
            
            yield train_mask, test_mask

In [21]:
p = 5
n_folds = 3
cv = UsersKFoldPOut(n_folds=n_folds, p=p)

for i, (train_mask, test_mask) in enumerate(cv.split(data)):
    train = data[train_mask]
    test = data[test_mask]
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 26284, Test: 835
Fold#1 | Train: 26284, Test: 835
Fold#2 | Train: 26291, Test: 830


In [22]:
intersect = set(train['user'].unique()).intersection(test['user'].unique())
intersect

set()